In [1]:
import numpy as np
import pandas as pd
import math


from pprint import pprint


                     

#Implemented by Raunak Rathour


df =pd.read_csv('heart.csv')
df.sort_values(by =['slope'],inplace = True)
#print(df)


df1 = df[['slope', 'num']]
data_heart =df1

#da = np.column_stack((X,Y))
#print(da)

In [3]:


def entro_py(target_col):
 
    elements,cnt = np.unique(target_col,return_counts = True)
    entro_py = np.sum([(-cnt[i]/np.sum(cnt))*np.log2(cnt[i]/np.sum(cnt)) for i in range(len(elements))])
    return entro_py

def Information_gain_(data,splt_attribute_name,target_name="num"):
  
    total_entro_py = entro_py(data[target_name])
    vals,cnt= np.unique(data[splt_attribute_name],return_counts=True)
    Weighted_Entropy = np.sum([(cnt[i]/np.sum(cnt))*entro_py(data.where(data[splt_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    Information_Gain = total_entro_py - Weighted_Entropy
    return Information_Gain

def I_D_3_algo(data,originaldata,attributes,target_attribute_name="num",parent_node_class = None):
 
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    elif len(data)==0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
    
    elif len(attributes) ==0:
        return parent_node_class
    
    else:
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        
        item_values = [Information_gain_(data,attribute,target_attribute_name) for attribute in attributes] #Return the information gain values for the attributes in the data_heart
        best_attribute_index = np.argmax(item_values)
        best_attribute = attributes[best_attribute_index]
        treee = {best_attribute:{}}

        attributes = [i for i in attributes if i != best_attribute]

        for value in np.unique(data[best_attribute]):
            value = value
            sub_data = data.where(data[best_attribute] == value).dropna()
            subtreee = I_D_3_algo(sub_data,data_heart,attributes,target_attribute_name,parent_node_class)
            treee[best_attribute][value] = subtreee    
        return(treee)    

def predict(query,treee,default = 1):
    
    for key in list(query.keys()):
        if key in list(treee.keys()):
            try:
                result = treee[key][query[key]] 
            except:
                return default
  
            result = treee[key][query[key]]
            if isinstance(result,dict):
                return predict(query,result)

            else:
                return result

        
        

def train_test_splt(data_heart):
    Train = data_heart.iloc[:100].reset_index(drop=True)
    Test = data_heart.iloc[100:].reset_index(drop=True)
    return Train,Test

Train = train_test_splt(data_heart)[0]
Test = train_test_splt(data_heart)[1] 



def test(data,treee):
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    predicted = pd.DataFrame(columns=["predicted"]) 
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],treee,1.0) 
    print('The accuracy is: ',(np.sum(predicted["predicted"] == data["num"])/len(data))*100,'%')
    


treee = I_D_3_algo(Train,Train,Train.columns[:-1])
print(" Tree will be like ")
pprint(treee)

test(Test,treee)

 Tree will be like 
{'slope': {1.0: 0}}
The accuracy is:  66.52719665271967 %
